# Adding Suggestions
This notebook will walk you through adding suggestions to LightTag. 

First, we do some basic imports and set up config

In [1]:
import requests
import json
import pandas as pd
from pprint import pprint
from requests.auth import HTTPBasicAuth

SERVER = "https://api-demo.lighttag.io" #The server is https://api-{your_subdomain_name.lighttag.io}
LT_USERNAME = "test" # Username of manager user
LT_PASSWORD = "test" #password of manager user

response = requests.post(SERVER +"/auth/token/create/",
              json={"username":LT_USERNAME,"password":LT_PASSWORD}
             )
response

<Response [200]>

Once we log in, LightTag will return a token we can use for further calls. We'll put that in dict that we will pass as header

In [9]:
token = response.json()['key']
headers={'Authorization': 'Token {token}'.format(token=token)}


### Getting the data to annotate
To generate suggestions we will need data. Particularly we'll need both all of the examples in a dataset as well as the tags from a particular schema. 
While you could gather each of these seperatly, LightTag has a data download view which already returns both of these combined. 
#### Warning
*At the moment the LightTag REST API is not fully documented, we are working to make it more consistent and document before releasing it publically. *



### Step 1: Get the Taskdefinition ID
As in the image below, press on Manager=>Project. You'll see a list of the tasks you have defined. Find the relevant task and press *Copy ID* to get it's id into the clipboard

![img](./copy_task_id.gif)

### Step 2: Prepare the http request
Paste the vaue you just copied into the variable *taskDefinitionId*. We'll use this as the get paramater which will retreive relevant data

In [27]:
# Step 1, build the URL
URL = "{server}/{endpoint}".format(server=SERVER,endpoint="manager/annotations/taskdefinition/")
taskDefinitionId="a4b83e46-e1b0-45fc-a974-2c5878715aea" #paste the id here
payload = {"taskDefinitionId":taskDefinitionId}

In [28]:
#Send the request
resp =requests.get(URL,params=payload,headers=headers
             )
resp.status_code

200

In [29]:
resp.request.path_url

'/manager/annotations/taskdefinition/?taskDefinitionId=a4b83e46-e1b0-45fc-a974-2c5878715aea'

### Step 3: Extract the dataset and the schema
The server will respond with a single dict that has our dataset (including ids) and schema (with Tags). We'll extract those seperately

In [30]:
data = resp.json() #Parse the response JSON into a python dict
schema = data['schema'] #extract the schema
tags = {x['name']:x['id'] for x in schema['tags']} #Make a map from tag name to id. 
#This will make defining suggestions easier
examples= data['dataset']['examples'] #Extract the list of examples

In [34]:
# Leys peek at the first example
pprint(examples[0])

{'annotations': [],
 'content': ' Now there was a certain man of Ramathaimzophim, of mount '
            'Ephraim, and his name was Elkanah, the son of Jeroham, the son of '
            'Elihu, the son of Tohu, the son of Zuph, an Ephrathite: ',
 'id': '7ed5886d1759423492d5152396b50f10',
 'metadata': {'book': 'The First Book of the Kings',
              'chapter': 1.1,
              'verse': 1}}


In [36]:
pprint(tags) # Print our dictionary of tags

{'Action': '16638acf-bc1b-471c-a535-3378913ab872',
 'Emotion': '01428033-dc8b-4dcc-8929-2622fde2e4bb',
 'God': '76f33848-dcbe-42ce-8587-185d1a722234',
 'Nation': '06432717-53d5-416f-9181-7420666573c5',
 'Object': '97b4540d-ebd8-447c-ab51-b41ebe5fd606',
 'Pagan God': '6c818c42-7901-4b39-9183-8e27764bd89b',
 'Person': '22bc9390-ea2d-4b88-9745-a2f4e65a4699',
 'Place': '1650c780-26e2-44f9-af3c-6b1e06cb95ed'}


### Step 4: Create your suggestions
How you create a suggestion is up to you. Youa re free to use a dicitonary, regex, neural network or whatever else
is availble. 
In this example we'll use the [flashtext](https://github.com/vi3k6i5/flashtext) library to mark all apearences of the word Lord as god and all appearences of the word Abraham as a Person


In [37]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword("LORD",tags["God"])
keyword_processor.add_keyword("Abraham",tags["Person"])

True

In [39]:
keyword_processor.extract_keywords(" In the Bible Abraham met the LORD one day|",
                                   span_info=True)

[('22bc9390-ea2d-4b88-9745-a2f4e65a4699', 14, 21),
 ('76f33848-dcbe-42ce-8587-185d1a722234', 30, 34)]

### Step 4.1: Iterate over your examples and make a  list of suggestions


In [40]:
suggestions = []
for example in examples: #(Notice the text is located in the content field)
    #For every suggestion that comes out of our model (in this case, flashtext)
    for tag_id,start,end in keyword_processor.extract_keywords(example['content'],span_info=True):
        suggestion= { #Create a suggestion
            "example_id":example['id'], #That refers to a particular example
            "tag_id":tag_id, #and applies a particular tag
            "start":start, #Which starts somewhere in the example
            "end":end # And ends somewhere in the example
        }
        suggestions.append(suggestion)

In [43]:
pprint(suggestions[:3])

[{'end': 86,
  'example_id': '6b8422156902463e8e89d73d50398bb8',
  'start': 82,
  'tag_id': '76f33848-dcbe-42ce-8587-185d1a722234'},
 {'end': 176,
  'example_id': '6b8422156902463e8e89d73d50398bb8',
  'start': 172,
  'tag_id': '76f33848-dcbe-42ce-8587-185d1a722234'},
 {'end': 76,
  'example_id': 'ad806446a8a14c96b8ecde61fec65d3a',
  'start': 72,
  'tag_id': '76f33848-dcbe-42ce-8587-185d1a722234'}]


### Step 5: Define a model and send the data

In [44]:
model_metadata= { # Define any metadata you'd like to store about the model
    "defined_by": "LightTag",
    "faith": -5
}
data = {
    "model":{
        "name":"demo_suggestions",  #Give the model a name
        "schema_id":schema["id"], # Specify which schema it relates to
        "metadata": model_metadata # Provide metadata (optional)
    },
    "suggestions":suggestions #Attatch the suggestions you made before
}

In [45]:
resp =requests.post(SERVER + "/manager/suggestions_model",
              headers=headers,
                    json=data,
                    
             )


In [200]:
resp

<Response [201]>